In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from pathlib import Path
import math

In [3]:
label_alpaca = '/m/0pcr'

In [4]:
train_set = pd.read_csv(r'c:\CodigoAlfredo\Train_Images_Open_image\oidv6-train-annotations-bbox.csv')

In [5]:
validation_set = pd.read_csv(r'c:\CodigoAlfredo\Validation_Image_Open_Image\validation-annotations-bbox.csv')

In [6]:
test_set = pd.read_csv(r'c:\CodigoAlfredo\Test_Image_Open_Image\test-annotations-bbox.csv')

In [8]:
alpacas_train = train_set.loc[train_set['LabelName'] == label_alpaca]
alpacas_test = test_set.loc[test_set['LabelName'] == label_alpaca]
alpacas_validation = validation_set.loc[validation_set['LabelName'] == label_alpaca]

In [9]:
a = alpacas_train.loc[alpacas_train['ImageID'] == '00ad17d214e32d2b']
a

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,...,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y
98033,00ad17d214e32d2b,xclick,/m/0pcr,1,0.000000,0.999375,0.106667,0.999167,1,1,...,0,0,0.689375,0.999375,0.595000,0.000000,0.106667,0.602500,0.999167,0.655833
98034,00ad17d214e32d2b,xclick,/m/0pcr,1,0.183125,0.620000,0.179167,0.999167,1,1,...,0,0,0.330625,0.183125,0.415625,0.620000,0.179167,0.508333,0.999167,0.444167
98035,00ad17d214e32d2b,xclick,/m/0pcr,1,0.646875,0.999375,0.104167,0.999167,1,1,...,0,0,0.687500,0.999375,0.906875,0.646875,0.104167,0.577500,0.999167,0.445000


In [10]:
print("Train: ")
print(len(alpacas_train))
print(alpacas_train['ImageID'].nunique(),'\n')
print("Test: ")
print(len(alpacas_test))
print(alpacas_test['ImageID'].nunique() , '\n')
print("Val: ")
print(len(alpacas_validation))
print(alpacas_validation['ImageID'].nunique())

Train: 
829
452 

Test: 
134
72 

Val: 
35
27


Escrevendo as identificações:

In [ ]:
# arquivo = Path("image_ids.txt")

# with arquivo.open('w') as f:
#     for img in alpacas_train.__array__():
#         f.write(f'train/{img[0]}\n')
#     for img in alpacas_test.__array__():
#         f.write(f'test/{img[0]}\n')
#     for img in alpacas_validation.__array__():
#         f.write(f'validation/{img[0]}\n')


In [11]:
Path(r'C:\CodigoAlfredo\datasetObjectDetection\images\train').mkdir(exist_ok=True, parents=True)
Path(r'C:\CodigoAlfredo\datasetObjectDetection\images\val').mkdir(exist_ok=True, parents=True)
Path(r'C:\CodigoAlfredo\datasetObjectDetection\labels\train').mkdir(exist_ok=True, parents=True)
Path(r'C:\CodigoAlfredo\datasetObjectDetection\labels\val').mkdir(exist_ok=True, parents=True)

In [12]:
lista_alpcas_ids = []
images = Path(r'C:\CodigoAlfredo\Images_Alpacas')

train_size = math.floor(len(list(images.rglob('*')))*.8)
test_size = math.ceil(len(list(images.rglob('*')))*.2)

train_img_path = Path(r'C:\CodigoAlfredo\datasetObjectDetection\images\train')
val_img_path = Path(r'C:\CodigoAlfredo\datasetObjectDetection\images\val')

label_train_path = Path(r'C:\CodigoAlfredo\datasetObjectDetection\labels\train')
label_val_path = Path(r'C:\CodigoAlfredo\datasetObjectDetection\labels\val')

In [16]:
def limpar_dados():
    for item in  Path(r'c:\CodigoAlfredo\datasetObjectDetection').iterdir():
        for sub in item.iterdir():
            for arq in sub.iterdir():
                arq.unlink()

In [17]:
limpar_dados()

In [18]:
def find_alpaca(id):

    for df in [alpacas_train,alpacas_test, alpacas_validation]:
        labels = []
        alpacas = df.loc[df['ImageID'] == id]
        if not alpacas.empty:

            for _, row in alpacas.iterrows():
                classe = 0
                centro_x = (row['XMin'] + row['XMax']) / 2
                centro_y = (row['YMin'] + row['YMax']) / 2
                width = row['XMax'] - row['XMin']
                height = row['YMax'] - row['YMin']

                labels.append(f"{classe} {centro_x} {centro_y} {width} {height}")
            
            return labels
    


In [19]:
train_count = len(list(train_img_path.rglob('*')))
for img in images.rglob('*'):
    # Id da imagem
    id = img.stem

    if(train_count < train_size):
        dest_img = train_img_path/ img.name
        dest_label = label_train_path/ f'{id}.txt'
        train_count += 1
    else:
        dest_img = val_img_path/ img.name
        dest_label = label_val_path/ f'{id}.txt'
    
    im = cv2.imread(str(img))
    cv2.imwrite(str(dest_img), im)

    labels = find_alpaca(id)
    if labels:
        with dest_label.open('w',encoding='utf-8') as f:
            for label in labels:
                f.write(f'{label}\n')

In [20]:
print(alpacas_train.iloc[0])


ImageID        000151d10b95b4d3
Source                   xclick
LabelName               /m/0pcr
Confidence                    1
XMin                   0.105625
XMax                   0.999375
YMin                   0.084586
YMax                    0.99906
IsOccluded                    0
IsTruncated                   1
IsGroupOf                     0
IsDepiction                   0
IsInside                      0
XClick1X                  0.325
XClick2X               0.999375
XClick3X               0.643125
XClick4X               0.105625
XClick1Y               0.084586
XClick2Y               0.783835
XClick3Y                0.99906
XClick4Y               0.299812
Name: 2346, dtype: object


In [21]:
from pathlib import Path
import cv2

broken = []

for img in Path("C:/CodigoAlfredo/datasetObjectDetection/images/train").rglob("*"):
    im = cv2.imread(str(img))
    if im is None:
        broken.append(img)

broken[:10], len(broken)


([], 0)